In [ ]:
from itertools import combinations, chain

# Prints out a set nicely
# names is an optional list of the names for each of the (integer) items
def settostr(s, names=None):
    if names is None:
        elems = [str(e) for e in s]
    else:
        elems = [names[e] for e in s]
    return "{" + (", ".join(elems)) + "}"

# Loads data from a file
def loaddata(filename):
    with open(filename) as f:
        nitems = int(f.readline())
        names = [f.readline().strip() for i in range(nitems)]
        nrows = int(f.readline())
        data = [[int(s) for s in f.readline().split()] for i in range(nrows)]
        f.close()
        return names, data, nitems

def learnrules(numitems, data, minsupport, minconfidence):
    # Generate frequent itemsets using the Apriori algorithm
    def apriori(data, minsupport):
        itemsets = [frozenset([item]) for item in range(numitems)]
        frequent_itemsets = []
        while itemsets:
            # Generate candidate itemsets
            candidates = set()
            for itemset in itemsets:
                for item in range(itemset[-1] + 1, numitems):
                    candidates.add(itemset | frozenset([item]))
            # Count support for each candidate itemset
            support_counts = {}
            for transaction in data:
                for candidate in candidates:
                    if candidate.issubset(transaction):
                        support_counts[candidate] = support_counts.get(candidate, 0) + 1
            # Prune candidates based on minimum support
            itemsets = []
            for candidate, support in support_counts.items():
                if support >= minsupport:
                    itemsets.append(candidate)
                    frequent_itemsets.append((candidate, support))
            # Join itemsets for the next iteration
            itemsets = list(chain(*[combinations(itemset, r + 2) for itemset in itemsets]))
        return frequent_itemsets

    # Generate association rules from frequent itemsets
    def generate_rules(frequent_itemsets, minconfidence):
        rules = []
        for itemset, support in frequent_itemsets:
            if len(itemset) >= 2:
                for i in range(1, len(itemset)):
                    for antecedent in combinations(itemset, i):
                        antecedent = frozenset(antecedent)
                        consequent = itemset - antecedent
                        confidence = support / frequent_itemsets_dict[antecedent]
                        if confidence >= minconfidence:
                            rules.append((antecedent, consequent, support, confidence))
        return sorted(rules, key=lambda x: x[3], reverse=True)

    frequent_itemsets = apriori(data, minsupport)
    frequent_itemsets_dict = {itemset: support for itemset, support in frequent_itemsets}
    rules = generate_rules(frequent_itemsets, minconfidence)
    return rules

def writerules(rules, data, itemnames):
    for antecedent, consequent, support, confidence in rules:
        antecedent_str = settostr(antecedent, itemnames)
        consequent_str = settostr(consequent, itemnames)
        support_str = "{:7.4f}".format(support / len(data))
        confidence_str = "{:7.4f}".format(confidence)
        print(f"{support_str} {confidence_str} {antecedent_str} => {consequent_str}")

def printruleset(datasetfilename, minsupport, minconfidence):
    itemnames, data, numitems = loaddata(datasetfilename)
    rules = learnrules(numitems, data, minsupport, minconfidence)
    writerules(rules, data, itemnames)

In [ ]:
#pseudocode idea

function Apriori-Gen(Li−1)
▷ From frequent items sets of size i − 1, generate candidate frequent items sets of size i
Ci ← {}
for all I ∈ Li−1 do
for all J ∈ Li−1, J ̸ = I do
if |I S J| = i then
Ci ← Ci
S{I S J}
return Ci


function Apriori(I , D, smin)
▷ From items (I ), data (D), and smin, generate all frequent itemsets
L1 ← {{i} | i ∈ I ∧ s({i}) ≥ smin}
L ← L1
i ← 1
while |Li | > 0 do
i ← i + 1
Ci ← Apriori-Gen(Li−1) ▷ Ci are the blue and orange nodes on level i
Li ← {} ▷ Li (will be) the blue nodes on level i
for all c ∈ Ci do
if s(c) ≥ smin then
Li ← Li
S{c}
L ← L S Li
return L



In [ ]:
def learnrules(numitems, data, minsupport, minconfidence):
    # Calculate support for each item
    item_support = {}
    for transaction in data:
        for item in transaction:
            if item in item_support:
                item_support[item] += 1
            else:
                item_support[item] = 1

    # Calculate frequent itemsets
    frequent_itemsets = []
    for item, support in item_support.items():
        if support >= minsupport:
            frequent_itemsets.append((frozenset([item]), support))

    # Generate association rules
    rules = []
    frequent_itemsets_dict = dict(frequent_itemsets)  # Convert frequent itemsets to a dictionary
    for k, support in frequent_itemsets:
        if len(k) > 1:
            generate_rules(k, [], support, frequent_itemsets_dict, rules, minconfidence, numitems)

    return rules


def generate_rules(itemset, prefix, support, frequent_itemsets, rules, minconfidence, numitems):
    for item in itemset:
        remaining = itemset - frozenset([item])
        confidence = support / frequent_itemsets[frozenset(prefix)]
        if confidence >= minconfidence:
            rules.append((prefix, remaining, support, confidence))
        if len(remaining) > 1:
            generate_rules(remaining, prefix + [item], support, frequent_itemsets, rules, minconfidence, numitems)


def writerules(rules, data, itemnames):
    # Sort rules by confidence in descending order
    sorted_rules = sorted(rules, key=lambda x: x[3], reverse=True)

    # Print each rule in the desired format
    for antecedent, consequent, support, confidence in sorted_rules:
        antecedent_names = [itemnames[item] for item in antecedent]
        consequent_names = [itemnames[item] for item in consequent]

        support_str = "{:7.4f}".format(support)
        confidence_str = "{:7.4f}".format(confidence)

        rule_str = "Support: {}, Confidence: {}, Rule: {} -> {}".format(support_str, confidence_str, antecedent_names, consequent_names)
        print(rule_str)


def printruleset(datasetfilename, minsupport, minconfidence):
    (itemnames, data, numitems) = loaddata(datasetfilename)
    rules = learnrules(numitems, data, minsupport, minconfidence)
    writerules(rules, data, itemnames)


# Example usage
printruleset('/usr/local/cs171/toymovies.txt', 0.3, 0.5)